Inspiration: https://www.kaggle.com/code/maryamnoroozi68/object-detection-by-using-yolov8/notebook

**Purpose**: to object detection in images by using **YOLOv8**.

**Object detection** is a computer technology related to computer vision and image processing that deals with detecting instances of semantic objects of a certain class (such as humans, buildings, or cars) in digital images and videos.
When it comes to object detection, popular detection frameworks are

* YOLO
* SSD
* Faster R-CNN

**YOLO** (You Only Look Once) is a method / way to do object detection. It is the algorithm /strategy behind how the code is going to detect objects in the image.

Earlier detection frameworks, looked at different parts of the image multiple times at different scales and repurposed image classification technique to detect objects. This approach is slow and inefficient.

YOLO takes entirely different approach. It looks at the entire image only once and goes through the network once and detects objects. Hence the name. It is very fast. That’s the reason it has got so popular.

In [2]:
import numpy as np
import pandas as pd
import cv2

from sklearn.utils import shuffle
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

import warnings

warnings.simplefilter('ignore')

In [8]:
df = pd.read_csv("C:/Users/Vladimir-adm/DATA/Data from 360 video Subsea/annotations/via_project_28Feb2024_9h11m_csv.csv")
# df = shuffle(df)
df

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,subsea-frame1(0).png,2912413,"{""rust"":""rust""}",3,0,"{""name"":""rect"",""x"":254,""y"":8,""width"":110,""heig...","{""name"":""weld""}"
1,subsea-frame1(0).png,2912413,"{""rust"":""rust""}",3,1,"{""name"":""rect"",""x"":476,""y"":6,""width"":104,""heig...","{""name"":""rope""}"
2,subsea-frame1(0).png,2912413,"{""rust"":""rust""}",3,2,"{""name"":""rect"",""x"":4,""y"":1261,""width"":1309,""he...","{""name"":""splash_zone""}"
3,subsea-frame1(1).png,2689182,"{""rust"":""rust""}",1,0,"{""name"":""rect"",""x"":2,""y"":1243,""width"":1311,""he...","{""name"":""splash_zone""}"
4,subsea-frame1(2).png,2496000,"{""rust"":""rust""}",0,0,{},{}
...,...,...,...,...,...,...,...
328,subsea-frame7101(3).png,2107145,"{""rust"":""rust""}",0,0,{},{}
329,subsea-frame7201(0).png,2042227,"{""rust"":""rust""}",1,0,"{""name"":""polygon"",""all_points_x"":[546,584,625,...","{""name"":""rope""}"
330,subsea-frame7201(1).png,2010226,"{""rust"":""rust""}",0,0,{},{}
331,subsea-frame7201(2).png,1762572,"{""rust"":""rust""}",0,0,{},{}


In [13]:
class_names = df.region_attributes.unique()
classes = {}
for c_name in class_names:
    if c_name == '{}':
        classes[0] = "No class"
    elif c_name == '{"name":"weld"}':
        classes[1] = "weld"
    elif c_name == '{"name":"cw"}':
        classes[2] = "cw"
    elif c_name == '{"name":"rope"}':
        classes[3] = "rope"
    elif c_name == '{"name":"splash_zone"}':
        classes[4] = "splash zone"

labels = dict(sorted(classes.items()))
print(labels)

{0: 'No class', 1: 'weld', 2: 'cw', 3: 'rope', 4: 'splash zone'}


In [15]:
df = df.reset_index()  # make sure indexes pair with number of rows
boxes_dict = {}

for index, row in df.iterrows():
    #print(row['filename'], row['region_shape_attributes'])
    boxes_dict[row['filename']] = eval(row['region_shape_attributes'])

print(boxes_dict)

{'subsea-frame1(0).png': {'name': 'rect', 'x': 4, 'y': 1261, 'width': 1309, 'height': 56}, 'subsea-frame1(1).png': {'name': 'rect', 'x': 2, 'y': 1243, 'width': 1311, 'height': 71}, 'subsea-frame1(2).png': {}, 'subsea-frame1(3).png': {}, 'subsea-frame101(0).png': {'name': 'rect', 'x': 480, 'y': 6, 'width': 96, 'height': 1311}, 'subsea-frame101(1).png': {}, 'subsea-frame101(2).png': {}, 'subsea-frame101(3).png': {}, 'subsea-frame201(0).png': {'name': 'rect', 'x': 657, 'y': 1270, 'width': 657, 'height': 50}, 'subsea-frame201(1).png': {'name': 'rect', 'x': 4, 'y': 1256, 'width': 1316, 'height': 63}, 'subsea-frame201(2).png': {'name': 'rect', 'x': 0, 'y': 1287, 'width': 465, 'height': 35}, 'subsea-frame201(3).png': {}, 'subsea-frame301(0).png': {'name': 'rect', 'x': 471, 'y': 6, 'width': 123, 'height': 1312}, 'subsea-frame301(1).png': {'name': 'rect', 'x': 8, 'y': 1001, 'width': 1310, 'height': 71}, 'subsea-frame301(2).png': {'name': 'rect', 'x': 6, 'y': 1024, 'width': 1312, 'height': 100},